In [8]:
def find_min(matrix:list):
    ans = float('inf')
    for row in matrix:
        for entry in row:
            if entry != 0 and entry < ans:
                ans = entry
    return ans

# row: away , column : home

In [9]:
import random
import copy

class Node():
    def __init__(self,matrix:list,dis,week,tracking_pos) -> None:
        self.matrix = matrix
        self.dis = dis
        self.week = week
        self.tracking_pos = tracking_pos
                        

def get_sub_node(distance_matrix,node:Node):
        """ returns the list contains all tuples: (matrix,pos,dis) """
        ans_matrix = []
        final_ans = []

        queue = [(node,0)]

        current_week = node.week + 1
        N = len(distance_matrix)
        assert N%2 == 0, "The number of teams must be even"
        
        while queue:
            temp,match = queue.pop()
            matrix = temp.matrix
            pos = temp.tracking_pos
            dis = temp.dis
            if match == N//2:
                if temp.matrix not in ans_matrix:
                    ans_matrix.append(temp.matrix)
                    final_ans.append((temp.matrix,temp.tracking_pos,temp.dis))
            else:
                for i in range(N):
                    if current_week not in matrix[i] and current_week not in [matrix[j][i] for\
                        j in range(N)]:
                        for j in range(N):
                            if i!=j and current_week not in matrix[j] and current_week not \
                                in [matrix[i][j] for i in range(N)]:
                                if matrix[i][j] == None:
                                    sub_matrix = copy.deepcopy(matrix)
                                    sub_matrix[i][j] = current_week
                                    sub_pos = copy.deepcopy(pos)
                                    sub_pos[i],sub_pos[j] = j,j
                                    sub_dis = dis + distance_matrix[pos[i]][j] + distance_matrix[pos[j]][j]
                                    sub_Node = Node(sub_matrix,sub_dis,current_week,sub_pos)
                                    queue.append((sub_Node,match+1))
        return final_ans

def backtrack_solve(input:list):
    N = len(input)
    queue = []
    ## tracking the best value in the tree
    best_value = float('inf')
    
    best_config = None
    ##
    ##initialize the root of the tree.
    root_matrix = [[None for i in range(N)] for i in range(N)]
    for i in range(N):
        root_matrix[i][i] = 0
    root_dis = 0
    week = 0
    root_tracking_pos = [i for i in range(N)]
    root = Node(root_matrix,root_dis,week,root_tracking_pos)
    queue.append(root)
    while queue:
        temp = queue.pop()
        assert isinstance(temp, Node)

        current_week = temp.week
        if current_week == 2*N - 2:
            if temp.dis < best_value:
                best_config = temp.matrix
                best_value = temp.dis
                print(f"Found better configuration, the new best value is : {best_value}")
                print(f"The new best config is {best_config}")
                print()

        else:
            next_week = current_week + 1
            list_of_nodes = get_sub_node(input,temp)
            for item in list_of_nodes:
                sub_matrix,sub_pos,sub_dis = item
                if sub_dis < best_value:
                    sub_Node = Node(sub_matrix,sub_dis,next_week,sub_pos)
                    queue.append(sub_Node)

# distance_matrix = [[random.randint(0,10) for i in range(4)] for i in range(4)]
# for i in range(4):
#     distance_matrix[i][i] = 0
#     for j in range(4):
#         distance_matrix[j][i] = distance_matrix[i][j]

distance_matrix = [[0, 2, 3, 10], [2, 0, 3, 6], [3, 3, 0, 9], [10, 6, 9, 0]]

print(distance_matrix)
print()
backtrack_solve(input=distance_matrix)


[[0, 2, 3, 10], [2, 0, 3, 6], [3, 3, 0, 9], [10, 6, 9, 0]]

Found better configuration, the new best value is : 79
The new best config is [[0, 3, 2, 1], [6, 0, 1, 2], [5, 4, 0, 3], [4, 5, 6, 0]]

Found better configuration, the new best value is : 75
The new best config is [[0, 3, 1, 2], [6, 0, 2, 1], [5, 4, 0, 3], [4, 5, 6, 0]]

Found better configuration, the new best value is : 71
The new best config is [[0, 3, 1, 2], [6, 0, 5, 1], [4, 2, 0, 3], [5, 4, 6, 0]]



In [10]:
### Greedy Search ###
def greedy_solve(input:list):
    N = len(input)
    queue = []
    ##initialize the root of the tree.
    root_matrix = [[None for i in range(N)] for i in range(N)]
    for i in range(N):
        root_matrix[i][i] = 0
    root_dis = 0
    week = 0
    root_tracking_pos = [i for i in range(N)]
    root = Node(root_matrix,root_dis,week,root_tracking_pos)
    queue.append(root)
    while queue:
        temp = queue.pop()
        assert isinstance(temp, Node)

        current_week = temp.week
        if current_week == 2*N - 2:
            print(temp.matrix,temp.dis)
        else:
            next_week = current_week + 1
            list_of_nodes = get_sub_node(input,temp)
            #node: sub_matrix,sub_pos,sub_dis
            new_list = sorted(list_of_nodes,key=lambda x:x[2])
            sub_matrix,sub_pos,sub_dis = list_of_nodes[0]
            new_node = Node(sub_matrix,sub_dis,next_week,sub_pos)
            queue.append(new_node)

greedy_solve(distance_matrix)
     


[[0, 4, 5, 6], [1, 0, 6, 5], [2, 3, 0, 4], [3, 2, 1, 0]] 77


In [11]:
def beam_solve(input,k):
    N = len(input)
    queue = []
    ## tracking the best value in the tree
    c_min = find_min(input)

    ##
    ##initialize the root of the tree.
    root_matrix = [[None for i in range(N)] for i in range(N)]
    for i in range(N):
        root_matrix[i][i] = 0
    root_dis = 0
    week = 0
    root_tracking_pos = [i for i in range(N)]
    root = Node(root_matrix,root_dis,week,root_tracking_pos)
    queue.append(root)
    best = None
    
    while True:
        nodes = copy.deepcopy(queue) 
        queue = []
        if nodes[0].week == 2*N - 2:
            for node in nodes:
                if best == None:
                    best = node.matrix,node.dis
                else:
                    if node.dis < best[1]:
                        best = node.matrix,node.dis
            break
        else:
            next_week = nodes[0].week + 1
            for node in nodes:
                lis = get_sub_node(input,node)
                sub_nodes = [Node(sub_matrix,sub_dis,next_week,sub_pos) for \
                    (sub_matrix,sub_pos,sub_dis) in lis]
                queue += sub_nodes
            queue = sorted(queue,key=lambda x: x.dis)[:k]

    print(best)

# distance_matrix = [[random.randint(0,10) for i in range(8)] for i in range(8)]
# for i in range(8):
#     distance_matrix[i][i] = 0
#     for j in range(8):
#         distance_matrix[j][i] = distance_matrix[i][j]
beam_solve(distance_matrix,10) 

([[0, 3, 4, 5], [6, 0, 5, 4], [1, 2, 0, 6], [2, 1, 3, 0]], 79)


In [111]:
import traceback
import random
import numpy 
class matrixNode():
    def __init__(self,matrix,week) -> None:
        self.matrix = matrix
        self.week = week
# def generate(N):
#     assert N%2 == 0, 'Number of teams must be even'
#     ## initilizr
#     initialize = [[None for i in range(N)] for i in range(N)]
#     for i in range(N):
#         initialize[i][i] = 0
#     week = 0
#     node = matrixNode(initialize,week)
    

#     #backtrack
#     def get_neigh(root_node:matrixNode):
#         ans_matrix = []
#         queue = [(root_node,0)]
#         N = len(root_node.matrix)
#         current_week = root_node.week + 1
#         assert N%2 == 0, "The number of teams must be even"
#         while queue:
#             temp,match = queue.pop()
#             temp:matrixNode
            
#             matrix = temp.matrix
#             if match == N//2:
#                 if temp.matrix not in ans_matrix:
#                     ans_matrix.append(temp)
#             else:
#                 for i in range(N):
#                     if current_week not in matrix[i] and current_week not in [matrix[j][i] for\
#                         j in range(N)]:
#                         for j in range(N):
#                             if i!=j and current_week not in matrix[j] and current_week not \
#                                 in [matrix[i][j] for i in range(N)]:
#                                 if matrix[i][j] == None:
#                                     sub_matrix = copy.deepcopy(matrix)
#                                     sub_matrix[i][j] = current_week
#                                     sub_node = matrixNode(sub_matrix,current_week)
#                                     queue.append((sub_node,match + 1))                  
#         return ans_matrix

    # while node.week != 2*N - 2:
    #     list_matrix = get_neigh(node)
    #     node = random.choice(list_matrix)
    # return node.matrix

def generate(N):

    mat_0 = list()
    for i in range(N):
        row = [None]*N
        row[i] = 0
        mat_0.append(row)

    all_coors_0 = list((a,b) for a in range(0,N) for b in range(0,N))
    for i in range(N):
        all_coors_0.remove((i,i))

    n_choices_0 = dict()
    choices_0 = dict()
    for i in range(1,N+1):
        n_choices_0[i] = N-1
        choices_0[i] = [a for a in range(1,N+1) if a != i]
    
    univ_tab = dict()

    univ_tab[0] = [copy.deepcopy(n_choices_0), copy.deepcopy(choices_0), copy.deepcopy(all_coors_0), copy.deepcopy(mat_0)]

    global result
    result = []

    def construct_mat(n):
        global result
        if n == 2*N - 2:
            result = univ_tab[n][3]
        else:
            nextweek = construct_week(n+1)
            if nextweek == False:
                construct_mat(n)
            elif nextweek == 'Reset':
                construct_mat(n-1)
            else:
                univ_tab[n+1] = copy.deepcopy(nextweek)
                construct_mat(n+1)

    # functions for contructing week 

    def create_priority_queue(n_choices_f):
        L = list()
        for a,b in n_choices_f.items():
            L.append((b,a))

        L.sort()

        Q = list()
        for item in L:
            Q.append(item[1])
        return Q

    def check_choices(choices_copy):
        L = list()
        for key in choices_copy.keys():
            if len(choices_copy[key]) == 0:
                L.append(key)
        for key in L:
            choices_copy.pop(key)

    def create_n_choices(choices):
        n_choices_f = dict()
        for key in choices.keys():
            n_choices_f[key] = len(choices[key])
        
        return n_choices_f
        
    def choose_match(a: int, n_choices: dict, choices: dict, choices_copy: dict, all_coors: list, rest: list, queue: list, mat: list, week: int, p_queue: list, b = None):
        
        if b == None:
            b = random.choice(choices_copy[a])

        rest.remove(a)
        rest.remove(b)
        queue.remove(a)
        queue.remove(b)
        p_queue = copy.deepcopy(queue)

        for key in choices_copy.keys():
            if a in choices_copy[key]:
                choices_copy[key].remove(a)
            if b in choices_copy[key]:
                choices_copy[key].remove(b)
        choices_copy.pop(a)
        choices_copy.pop(b)

        RC = []
        if (a-1,b-1) in all_coors:
            RC.append((a,b))
        if (b-1,a-1) in all_coors:
            RC.append((b,a))
        
        (c,d) = random.choice(RC)

        all_coors.remove((c-1,d-1))

        if len(RC) == 1:
            n_choices[a] -= 1
            n_choices[b] -= 1
            choices[a].remove(b)
            choices[b].remove(a)

        mat[c-1][d-1] = week
        
    # constructing week

    def construct_week(m):

        univ = copy.deepcopy(univ_tab[m-1])
        # n_choices = univ[0]
        # choices = univ[1]
        # all_coors = univ[2]
        # mat = univ[3]
        
        choices_copy = copy.deepcopy(univ[1])

        queue = create_priority_queue(univ[0])

        p_queue = queue[:]
        rest = [i for i in range(1,N+1)]

        while len(p_queue) >=1:

            if len(rest) <= 2:
                if len(rest) == 1:
                    return False
                else:
                    if rest[1] not in univ[1][rest[0]]:
                        return False
                    else:
                        check_choices(choices_copy)
                        choose_match(rest[1], univ[0], univ[1], choices_copy, univ[2], rest, queue, univ[3], m, p_queue, rest[0])
                        return univ
            else:
                check_choices(choices_copy)
                queue = create_priority_queue(create_n_choices(choices_copy))
                p_queue = queue[:]

                i = p_queue[0]

                # print('Week', end ='')
                # print(m)

                choose_match(i, univ[0], univ[1], choices_copy, univ[2], rest, queue, univ[3], m, p_queue)

                if len(choices_copy.keys()) % 2 != 0:
                    return 'Reset'

    try:
        construct_mat(0)
    except Exception:

        construct_mat(0)

    return result


def get_neighbors(initialize):
    mat,cost = initialize
    M = copy.deepcopy(mat)
    N = len(M)
    
    all_neighbors = list()

    def team_cost(team,matrix,d_matrix):
        away = list(zip(list(matrix[team][i] for i in range(0,N)),range(1,N+1)))
        home = list(zip(list(matrix[i][team] for i in range(0,N)),list(team + 1 for j in range(0,N))))
        home.sort()
        home.pop(0)
        team_schedule = away + home
        team_schedule.sort()
        C = 0
        for i in range(len(team_schedule)-1):
            C += d_matrix[team_schedule[i][1] - 1][team_schedule[i+1][1] - 1]
        return C
        
    def match_swap(coor):
        Y = copy.deepcopy(mat)
        temp = Y[coor[0]][coor[1]]
        Y[coor[0]][coor[1]] = Y[coor[1]][coor[0]]
        Y[coor[1]][coor[0]] = temp

        A = team_cost(coor[0],mat,distance_matrix) + team_cost(coor[1],mat,distance_matrix) 
        B = team_cost(coor[0],Y,distance_matrix) + team_cost(coor[1],Y,distance_matrix)
        cost_Y = cost - A + B
        return (Y, cost_Y)

    def team_swap(teams):
        a,b = teams
        a -= 1
        b -= 1
        Y = copy.deepcopy(mat)
        Y_1 = numpy.array(copy.deepcopy(Y))
        
        def replace_in_mat(x,y,S):
            temp = S[x].copy()
            S[x] = S[y].copy()
            S[y] = temp.copy()

            S[x][y] = S[x][x]
            S[x][x] = 0

            S[y][x] = S[y][y]
            S[y][y] = 0

        replace_in_mat(a,b,Y_1)
        Y_1 = Y_1.T.copy()
        
        replace_in_mat(a,b,Y_1)
        Y_1 = Y_1.T.copy()

        Y = copy.deepcopy(Y_1.tolist())
        return (Y, compute_cost(distance_matrix, Y))
    
    iterate = list((j,i) for i in range(0,N) for j in range(0,i))

    for coor in iterate:
        mat_1, cost_1 = copy.deepcopy(match_swap(coor))
        all_neighbors.append((mat_1, cost_1))


    iterate = list((i,j) for i in range(1,N + 1) for j in range(i, N + 1))

    for coor in iterate:
        mat_1, cost_1 = copy.deepcopy(team_swap(coor))
        all_neighbors.append((mat_1, cost_1))
    
    return all_neighbors


def compute_cost(distance_matrix,matrix):
    '''compute cost of a solution'''
    num_of_teams = len(matrix)
    cost = 0
    for team in range(num_of_teams):
        current_pos = team
        team_dis = 0
        current_week = 0
        while current_week < 2*num_of_teams - 2:
            current_week = current_week + 1
            flag = True
            for index in range(num_of_teams):
                if matrix[team][index] == current_week:
                    old_pos = current_pos
                    current_pos = index
                    team_dis += distance_matrix[old_pos][current_pos]
                    flag = False
                    break
            if flag: # da san nha
                old_pos = current_pos
                current_pos = team
                team_dis += distance_matrix[old_pos][current_pos]

        cost += team_dis
        
    return cost
    
def local_solve(input):
    # random inial
    #input: distance matrix
    #return the schedule: matrix , cost
    N = len(input)
    random_matrix = generate(N)
    cost = compute_cost(input,random_matrix)
    initialize = random_matrix,cost
    flag = False
    while flag:
        neighbors = get_neighbors(initialize)
        for neighbor in neighbors:
            #neighbor: matrix:schedule,cost
            schedule,cost = neighbor
            flag = False
            if cost < initialize[1]:
                initialize = neighbor
                flag = True
    return initialize[0],initialize[1]




In [140]:
distance_matrix = [[random.randint(0,100) for i in range(40)] for i in range(40)]
for i in range(40):
    distance_matrix[i][i] = 0
    for j in range(40):
        distance_matrix[j][i] = distance_matrix[i][j]
local_solve(distance_matrix)[1]

118457

In [142]:
distance_matrix

[[0,
  86,
  59,
  7,
  26,
  2,
  47,
  64,
  99,
  61,
  51,
  58,
  7,
  5,
  93,
  8,
  53,
  94,
  46,
  29,
  18,
  47,
  100,
  24,
  85,
  93,
  56,
  77,
  6,
  74,
  29,
  79,
  54,
  14,
  6,
  75,
  56,
  45,
  87,
  71],
 [86,
  0,
  68,
  19,
  74,
  70,
  15,
  56,
  7,
  14,
  68,
  4,
  31,
  7,
  75,
  78,
  10,
  39,
  75,
  92,
  79,
  75,
  12,
  0,
  76,
  88,
  44,
  58,
  55,
  14,
  1,
  84,
  64,
  68,
  96,
  64,
  60,
  63,
  25,
  55],
 [59,
  68,
  0,
  40,
  4,
  45,
  97,
  79,
  72,
  14,
  9,
  32,
  84,
  98,
  77,
  61,
  21,
  97,
  81,
  43,
  54,
  70,
  11,
  6,
  12,
  74,
  54,
  47,
  7,
  79,
  52,
  23,
  97,
  71,
  100,
  86,
  7,
  70,
  34,
  41],
 [7,
  19,
  40,
  0,
  43,
  93,
  16,
  42,
  7,
  37,
  81,
  5,
  39,
  41,
  2,
  86,
  17,
  34,
  26,
  30,
  57,
  57,
  30,
  70,
  69,
  69,
  6,
  100,
  61,
  3,
  70,
  64,
  9,
  38,
  21,
  38,
  36,
  52,
  44,
  11],
 [26,
  74,
  4,
  43,
  0,
  70,
  3,
  58,
  65,
  3,
  99,

In [141]:
def iterative_local_solve(input,K):
    best_config = None
    best_cost = float('inf')
    for i in range(K):
        matrix,cost = local_solve(input)
        if cost < best_cost:
            best_cost = cost
            best_config = matrix

    return best_config,best_cost
iterative_local_solve(distance_matrix,K=40)[1]

114249

In [ ]:
def gibb_sampling(input):
    